<a href="https://colab.research.google.com/github/RAVIKUMARBALIJA/tensorflow2_tutorials/blob/main/tf_hub_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow-hub
!pip install tensorflow-datasets


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as tfhub
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential,Model


In [5]:
print(tf.__version__)
print('Eager execution',tf.executing_eagerly())
print('hub version',tfhub.__version__)
print("GPU is", "Available" if tf.config.list_physical_devices('TPU') else "Not Available")

2.6.0
Eager execution True
hub version 0.12.0
GPU is Not Available


In [12]:
train_data,validation_data,test_data=tfds.load(name='imdb_reviews',split=('train[:60%]','train[60%:]','test'),as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteVBBMPO/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteVBBMPO/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [16]:
train_examples_batch, train_labels_batch=next(iter(train_data.batch(10)))

In [18]:
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer=tfhub.KerasLayer(embedding,trainable=True,input_shape=[],dtype=tf.string)

In [19]:
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 50), dtype=float32, numpy=
array([[ 0.5423195 , -0.0119017 ,  0.06337538,  0.06862972, -0.16776837,
        -0.10581174,  0.16865303, -0.04998824, -0.31148055,  0.07910346,
         0.15442263,  0.01488662,  0.03930153,  0.19772711, -0.12215476,
        -0.04120981, -0.2704109 , -0.21922152,  0.26517662, -0.80739075,
         0.25833532, -0.3100421 ,  0.28683215,  0.1943387 , -0.29036492,
         0.03862849, -0.7844411 , -0.0479324 ,  0.4110299 , -0.36388892,
        -0.58034706,  0.30269456,  0.3630897 , -0.15227164, -0.44391504,
         0.19462997,  0.19528408,  0.05666234,  0.2890704 , -0.28468323,
        -0.00531206,  0.0571938 , -0.3201318 , -0.04418665, -0.08550783,
        -0.55847436, -0.23336391, -0.20782952, -0.03543064, -0.17533456],
       [ 0.56338924, -0.12339553, -0.10862679,  0.7753425 , -0.07667089,
        -0.15752277,  0.01872335, -0.08169781, -0.3521876 ,  0.4637341 ,
        -0.08492756,  0.07166859, -0.00670817,  0.12686075, -0.19326553,
 

In [20]:
hub_layer(train_examples_batch[:3]).shape

TensorShape([3, 50])

In [21]:
model=Sequential()
model.add(hub_layer)
model.add(Dense(16,activation='relu')),
model.add(Dense(1,activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_1 (KerasLayer)   (None, 50)                48190600  
_________________________________________________________________
dense (Dense)                (None, 16)                816       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________


In [24]:
model.compile(optimizer='adam',loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),metrics=['accuracy'])

In [26]:
history=model.fit(train_data.shuffle(1000).batch(512),validation_data=validation_data.batch(512),epochs=5,verbose=1)

Epoch 1/5


/usr/local/lib/python3.7/dist-packages/keras/backend.py:4994: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`binary_crossentropy` received `from_logits=True`, but the `output`'


30/30 [==============================] - 21s 655ms/step - loss: 0.6600 - accuracy: 0.6181 - val_loss: 0.6134 - val_accuracy: 0.7104
Epoch 2/5
30/30 [==============================] - 19s 641ms/step - loss: 0.5509 - accuracy: 0.7764 - val_loss: 0.5161 - val_accuracy: 0.7840
Epoch 3/5
30/30 [==============================] - 19s 648ms/step - loss: 0.4250 - accuracy: 0.8504 - val_loss: 0.4185 - val_accuracy: 0.8308
Epoch 4/5
30/30 [==============================] - 19s 636ms/step - loss: 0.3097 - accuracy: 0.8979 - val_loss: 0.3588 - val_accuracy: 0.8525
Epoch 5/5
30/30 [==============================] - 19s 642ms/step - loss: 0.2269 - accuracy: 0.9284 - val_loss: 0.3280 - val_accuracy: 0.8600


In [28]:
results = model.evaluate(test_data.batch(512), verbose=2)

49/49 - 5s - loss: 0.3423 - accuracy: 0.8533


In [29]:
for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

loss: 0.342
accuracy: 0.853
